# M&A SAMYANG Report 

In [35]:
import pandas as pd
import numpy as np
from pathlib import Path
import re
from datetime import datetime

# ==========================
# 셀 1 — 파일 목록 & 비교 대상 자동 선택 (지역변수 + prefix)
# ==========================

sy_drayage_DATA_DIR = Path("data") / "SY" / "drayage"

sy_drayage_files = list(sy_drayage_DATA_DIR.glob("*.xlsx"))

def sy_drayage_extract_date(path: Path):
    m = re.match(r"(\d{8})", path.name)
    if not m:
        return None
    raw = m.group(1)
    try:
        return datetime.strptime(raw, "%m%d%Y").date()
    except ValueError:
        return None

sy_drayage_files_sorted = sorted(
    sy_drayage_files,
    key=lambda p: sy_drayage_extract_date(p) or datetime.min.date()
)

sy_drayage_prev_file = sy_drayage_files_sorted[-2]
sy_drayage_curr_file = sy_drayage_files_sorted[-1]

sy_drayage_prev_date = sy_drayage_extract_date(sy_drayage_prev_file)
sy_drayage_curr_date = sy_drayage_extract_date(sy_drayage_curr_file)

sy_drayage_today = datetime.today().date()

print(f"- Today        : {sy_drayage_today}")
print(f"- 최신 리포트     : {sy_drayage_curr_date}  ({sy_drayage_curr_file.name})")
print(f"- 이전 리포트     : {sy_drayage_prev_date}  ({sy_drayage_prev_file.name})")


- Today        : 2025-11-23
- 최신 리포트     : 2025-11-21  (11212025 - 2.xlsx)
- 이전 리포트     : 2025-11-20  (11202025 - 2.xlsx)


In [36]:
# ==========================
# 셀 2 — 전날/오늘 리포트 로드 + 공통 컬럼 정리 (지역변수 + prefix)
# ==========================

sy_drayage_KEY = ["Customer Reference No.", "Container No."]

sy_drayage_DISPLAY_COLS = [
    "Customer Reference No.",
    "Pick Up Location",
    "Master B/L No.",
    "Container No.",
    "Last Free Date",
    "P/U APPT DATE",
    "P/U APPT TIME",
    "EMPTY NOTICE",
    "Container Remark",
    "DELIVERY DATE",
    "Return Date",
]

# 1) 파일 읽기
sy_drayage_df_prev = pd.read_excel(sy_drayage_prev_file)
sy_drayage_df_curr = pd.read_excel(sy_drayage_curr_file)

# 2) 컬럼 공백/개행 제거
sy_drayage_df_prev.columns = sy_drayage_df_prev.columns.str.strip()
sy_drayage_df_curr.columns = sy_drayage_df_curr.columns.str.strip()

# 3) 필요 컬럼 추가 (없는 경우 NaN 생성)
for sy_drayage_col in sy_drayage_DISPLAY_COLS:
    if sy_drayage_col not in sy_drayage_df_prev.columns:
        sy_drayage_df_prev[sy_drayage_col] = pd.NA
    if sy_drayage_col not in sy_drayage_df_curr.columns:
        sy_drayage_df_curr[sy_drayage_col] = pd.NA

# 4) DISPLAY_COLS 순서대로 통일
sy_drayage_df_prev = sy_drayage_df_prev[sy_drayage_DISPLAY_COLS]
sy_drayage_df_curr = sy_drayage_df_curr[sy_drayage_DISPLAY_COLS]

print("셀2 완료 — 리포트 로드 및 컬럼 정리 완료")


셀2 완료 — 리포트 로드 및 컬럼 정리 완료


---

## Cell 3 - 일자별 요약 리포트

In [37]:
# ==========================
# sy_drayage 셀 3 — 변경 요약
# ==========================

import math
from datetime import datetime
import re
from collections import Counter

# 1) 인덱스 설정
sy_drayage_df_prev_k = sy_drayage_df_prev.set_index(sy_drayage_KEY)
sy_drayage_df_curr_k = sy_drayage_df_curr.set_index(sy_drayage_KEY)

sy_drayage_prev_idx = sy_drayage_df_prev_k.index
sy_drayage_curr_idx = sy_drayage_df_curr_k.index

# 2) 공통 컨테이너 / 신규 컨테이너
sy_drayage_common_idx    = sy_drayage_prev_idx.intersection(sy_drayage_curr_idx)
sy_drayage_curr_only_idx = sy_drayage_curr_idx.difference(sy_drayage_prev_idx)

# 3) 공통 컨테이너 비교 준비
sy_drayage_prev_c = sy_drayage_df_prev_k.loc[sy_drayage_common_idx]
sy_drayage_curr_c = sy_drayage_df_curr_k.loc[sy_drayage_common_idx]

sy_drayage_shared_cols = [c for c in sy_drayage_prev_c.columns if c in sy_drayage_curr_c.columns]

# strip 적용해서 비교용 복사
sy_drayage_prev_c2 = sy_drayage_prev_c[sy_drayage_shared_cols].copy()
sy_drayage_curr_c2 = sy_drayage_curr_c[sy_drayage_shared_cols].copy()

for col in sy_drayage_shared_cols:
    try:
        sy_drayage_prev_c2[col] = sy_drayage_prev_c2[col].astype(str).str.strip()
        sy_drayage_curr_c2[col] = sy_drayage_curr_c2[col].astype(str).str.strip()
    except:
        pass

# NaN 동일 처리
sy_drayage_equal_mask   = (sy_drayage_prev_c2 == sy_drayage_curr_c2) | (sy_drayage_prev_c2.isna() & sy_drayage_curr_c2.isna())
sy_drayage_changed_mask = ~sy_drayage_equal_mask

# 4) “컨테이너 1행 단위 변경 판단” = 6개만 체크
sy_drayage_important_cols = [
    "Last Free Date",
    "P/U APPT DATE",
    "P/U APPT TIME",
    "DELIVERY DATE",
    "Return Date",
    "Container Remark",
]
sy_drayage_important_cols = [c for c in sy_drayage_important_cols if c in sy_drayage_shared_cols]

sy_drayage_imp_mask = sy_drayage_changed_mask[sy_drayage_important_cols]
sy_drayage_changed_rows_count = int(sy_drayage_imp_mask.any(axis=1).sum())

# 5) 신규 컨테이너 수
sy_drayage_added_containers = len(sy_drayage_curr_only_idx)

# 6) 날짜/시간 컬럼 통계
sy_drayage_date_cols = ["Last Free Date", "P/U APPT DATE", "DELIVERY DATE", "Return Date"]
sy_drayage_date_cols = [c for c in sy_drayage_date_cols if c in sy_drayage_shared_cols]

sy_drayage_column_stats = []
for col in sy_drayage_date_cols + ["P/U APPT TIME"]:
    if col not in sy_drayage_shared_cols:
        continue
    prev_col = sy_drayage_prev_c2[col]
    curr_col = sy_drayage_curr_c2[col]

    added_values = int((prev_col.isna() & curr_col.notna()).sum())
    changed_cells = int(sy_drayage_changed_mask[col].sum())

    sy_drayage_column_stats.append({
        "Column": col,
        "Added_values": added_values,
        "Changed_cells": changed_cells,
    })

# Delivery(예정)
if "Container Remark" in sy_drayage_shared_cols:
    sy_drayage_prev_rem = sy_drayage_prev_c2["Container Remark"]
    sy_drayage_curr_rem = sy_drayage_curr_c2["Container Remark"]

    sy_drayage_delivery_added = int((sy_drayage_prev_rem.isna() & sy_drayage_curr_rem.notna()).sum())
    sy_drayage_delivery_changed = int(sy_drayage_changed_mask["Container Remark"].sum())
else:
    sy_drayage_delivery_added = 0
    sy_drayage_delivery_changed = 0

# 한글 표시명
sy_drayage_col_kor = {
    "Last Free Date":   "LFD",
    "P/U APPT DATE":    "픽업 날짜",
    "P/U APPT TIME":    "픽업 시간",
    "DELIVERY DATE":    "배송 완료일(Delivered)",
    "Return Date":      "리턴 날짜(Return)",
}

# helper
sy_drayage_stats_by_col = {x["Column"]: x for x in sy_drayage_column_stats}

def sy_drayage_get_counts(col_name):
    s = sy_drayage_stats_by_col.get(col_name, {"Changed_cells": 0, "Added_values": 0})
    return s["Changed_cells"], s["Added_values"]

lfd_changed, lfd_new = sy_drayage_get_counts("Last Free Date")
pu_date_changed, pu_date_new = sy_drayage_get_counts("P/U APPT DATE")
pu_time_changed, pu_time_new = sy_drayage_get_counts("P/U APPT TIME")
deliv_changed = sy_drayage_delivery_changed
deliv_new = sy_drayage_delivery_added
delivered_changed, delivered_new = sy_drayage_get_counts("DELIVERY DATE")
return_changed, return_new = sy_drayage_get_counts("Return Date")

# 출력
print("=== SY Drayage 리포트 변경 요약 ===")
print(f"- 신규 컨테이너 추가: {sy_drayage_added_containers} 건")
print(f"- 값이 변경된 컨테이너 행: {sy_drayage_changed_rows_count} 건\n")

print(f"- LFD: 변경 {lfd_changed} 건 (New: {lfd_new} 건)")
print(f"- 픽업 날짜: 변경 {pu_date_changed} 건 (New: {pu_date_new} 건)")
print(f"- 픽업 시간: 변경 {pu_time_changed} 건 (New: {pu_time_new} 건)\n")

print(f"- 배송 예정일(Delivery): 변경 {deliv_changed} 건 (New: {deliv_new} 건)")
print(f"- 배송 완료일(Delivered): 변경 {delivered_changed} 건 (New: {delivered_new} 건)")
print(f"- 리턴 날짜(Return): 변경 {return_changed} 건 (New: {return_new} 건)\n")

print("\n--- 날짜별 요약 ---")

# (원본 로직 동일 — LFD, 픽업 날짜 part)
# ----------------------------------------------------------
for col in ["Last Free Date", "P/U APPT DATE"]:
    if col not in sy_drayage_date_cols:
        continue

    prev_col = sy_drayage_prev_c2[col].dropna()
    curr_col = sy_drayage_curr_c2[col].dropna()

    if hasattr(prev_col, "dt"):
        prev_dates = prev_col.dt.date
    else:
        continue

    if hasattr(curr_col, "dt"):
        curr_dates = curr_col.dt.date
    else:
        continue

    prev_counts = prev_dates.value_counts()
    curr_counts = curr_dates.value_counts()

    all_dates = sorted(set(prev_counts.index).union(curr_counts.index))

    total_changed = 0
    lines = []
    for d in all_dates:
        prev_n = int(prev_counts.get(d, 0))
        curr_n = int(curr_counts.get(d, 0))
        if prev_n == curr_n:
            continue

        diff = curr_n - prev_n
        total_changed += abs(diff)
        sign = "+" if diff > 0 else ""
        lines.append(f"  - {d}: {prev_n} 건 → {curr_n} 건 ({sign}{diff} 건)")

    name = sy_drayage_col_kor.get(col, col)
    print(f"[{name}] 총 {total_changed} 건 변경")
    for line in lines:
        print(line)

# ----------------------------------------------------------
# Delivery / 예외 / Delivered / Return 날짜 요약
# (원본 로직 1:1 유지 — prefix만 적용)
# ----------------------------------------------------------

def sy_drayage_parse_delivery_cell(val, year):
    if pd.isna(val):
        return None, None, None
    if isinstance(val, (pd.Timestamp, datetime)):
        return val.date(), "DATE", None

    s = str(val).strip()
    if not s:
        return None, None, None

    up = s.upper()
    if up.startswith("MT"):
        return None, None, None

    date_matches = re.findall(r"(\d{1,2})/(\d{1,2})", up)

    if len(date_matches) >= 2 or "OR" in up:
        return None, None, s

    if len(date_matches) == 1 and "DEL" in up:
        mm, dd = date_matches[0]
        try:
            d = datetime(year, int(mm), int(dd)).date()
        except:
            return None, None, s
        tag = "DEL-D" if "DIRECT" in up else "DEL"
        return d, tag, None

    return None, None, s


def sy_drayage_build_delivery_series(rem_s):
    dates, tags, ex = [], [], []
    base_year = sy_drayage_curr_date.year

    for v in rem_s:
        d, t, e = sy_drayage_parse_delivery_cell(v, base_year)
        dates.append(d)
        tags.append(t)
        ex.append(e)

    return (
        pd.Series(dates, index=rem_s.index),
        pd.Series(tags, index=rem_s.index),
        pd.Series(ex, index=rem_s.index),
    )


if "Container Remark" in sy_drayage_shared_cols:
    sy_prev_dates, sy_prev_tags, sy_prev_ex = sy_drayage_build_delivery_series(sy_drayage_prev_rem)
    sy_curr_dates, sy_curr_tags, sy_curr_ex = sy_drayage_build_delivery_series(sy_drayage_curr_rem)

    all_dates = sorted(
        set(sy_prev_dates.dropna().unique()) |
        set(sy_curr_dates.dropna().unique())
    )

    print("[배송 예정일(Delivery)] 날짜 기준 변경 요약")
    for d in all_dates:
        prev_mask = sy_prev_dates == d
        curr_mask = sy_curr_dates == d

        prev_total = int(prev_mask.sum())
        curr_total = int(curr_mask.sum())
        diff = curr_total - prev_total

        if diff != 0:
            prev_del   = int((prev_mask & (sy_prev_tags == "DEL")).sum())
            prev_deld  = int((prev_mask & (sy_prev_tags == "DEL-D")).sum())
            curr_del   = int((curr_mask & (sy_curr_tags == "DEL")).sum())
            curr_deld  = int((curr_mask & (sy_curr_tags == "DEL-D")).sum())
            sign = "+" if diff > 0 else ""
            print(
                f"  - {d}: {prev_total} 건 → {curr_total} 건 ({sign}{diff} 건) "
                f"(DEL: {prev_del} → {curr_del}, DEL-D: {prev_deld} → {curr_deld})"
            )

    sy_prev_ex_counts = Counter(t for t in sy_prev_ex.dropna())
    sy_curr_ex_counts = Counter(t for t in sy_curr_ex.dropna())
    all_ex_keys = sorted(set(sy_prev_ex_counts.keys()) | set(sy_curr_ex_counts.keys()))

    if all_ex_keys:
        total_ex_changed = 0
        for key in all_ex_keys:
            if sy_prev_ex_counts.get(key, 0) != sy_curr_ex_counts.get(key, 0):
                total_ex_changed += abs(sy_curr_ex_counts.get(key, 0) - sy_prev_ex_counts.get(key, 0))

        print(f"[배송 일정 예외] : 총 {total_ex_changed}건 변경")
        for key in all_ex_keys:
            print(f"  '{key}': 이전 {sy_prev_ex_counts.get(key, 0)} 건, 오늘 {sy_curr_ex_counts.get(key, 0)} 건")

# Delivered / Return
for col in ["DELIVERY DATE", "Return Date"]:
    if col not in sy_drayage_date_cols:
        continue

    prev_col = sy_drayage_prev_c2[col].dropna()
    curr_col = sy_drayage_curr_c2[col].dropna()

    if hasattr(prev_col, "dt"):
        prev_dates = prev_col.dt.date
    else:
        continue
    if hasattr(curr_col, "dt"):
        curr_dates = curr_col.dt.date
    else:
        continue

    prev_counts = prev_dates.value_counts()
    curr_counts = curr_dates.value_counts()
    all_dates = sorted(set(prev_counts.index).union(curr_counts.index))

    total_changed = 0
    lines = []
    for d in all_dates:
        prev_n = int(prev_counts.get(d, 0))
        curr_n = int(curr_counts.get(d, 0))
        if prev_n != curr_n:
            diff = curr_n - prev_n
            total_changed += abs(diff)
            sign = "+" if diff > 0 else ""
            lines.append(f"  - {d}: {prev_n} 건 → {curr_n} 건 ({sign}{diff} 건)")

    name = sy_drayage_col_kor.get(col, col)
    print(f"[{name}] 총 {total_changed} 건 변경")
    for line in lines:
        print(line)


=== SY Drayage 리포트 변경 요약 ===
- 신규 컨테이너 추가: 0 건
- 값이 변경된 컨테이너 행: 65 건

- LFD: 변경 4 건 (New: 0 건)
- 픽업 날짜: 변경 18 건 (New: 0 건)
- 픽업 시간: 변경 16 건 (New: 0 건)

- 배송 예정일(Delivery): 변경 56 건 (New: 0 건)
- 배송 완료일(Delivered): 변경 18 건 (New: 0 건)
- 리턴 날짜(Return): 변경 17 건 (New: 0 건)


--- 날짜별 요약 ---
[배송 예정일(Delivery)] 날짜 기준 변경 요약
  - 2025-11-21: 15 건 → 0 건 (-15 건) (DEL: 15 → 0, DEL-D: 0 → 0)
  - 2025-11-22: 0 건 → 12 건 (+12 건) (DEL: 0 → 12, DEL-D: 0 → 0)
  - 2025-11-24: 9 건 → 8 건 (-1 건) (DEL: 1 → 0, DEL-D: 8 → 8)
  - 2025-11-25: 3 건 → 6 건 (+3 건) (DEL: 3 → 6, DEL-D: 0 → 0)
[배송 일정 예외] : 총 34건 변경
  'DEL 11/22 OR 11/24': 이전 0 건, 오늘 8 건
  'LINE HOLD ERROR': 이전 1 건, 오늘 0 건
  'SUPA DUPA URGENT': 이전 0 건, 오늘 1 건
  'nan': 이전 60 건, 오늘 36 건



## Cell 3 End

---

## Cell 4 - Final Report 

In [41]:
# ==========================
# sy_drayage 셀 4 — 변경된 컨테이너 상세 표
#  - strip() 기반 동일값은 변경 아님
#  - Delivery 파싱 동일 기준
#  - 빈칸 Delivery 는 색칠 안함
# ==========================

import numpy as np
from datetime import date
import re

# =========================================================
# 1) strip 비교 동일하게 적용된 prev/curr 준비
# =========================================================

sy_prev_k = sy_drayage_df_prev.set_index(sy_drayage_KEY)
sy_curr_k = sy_drayage_df_curr.set_index(sy_drayage_KEY)

sy_prev_idx = sy_prev_k.index
sy_curr_idx = sy_curr_k.index

sy_common_idx = sy_prev_idx.intersection(sy_curr_idx)

# subset (공통 row)
sy_prev_c = sy_prev_k.loc[sy_common_idx]
sy_curr_c = sy_curr_k.loc[sy_common_idx]

# 공통 컬럼만
sy_shared_cols = [c for c in sy_prev_c.columns if c in sy_curr_c.columns]

# strip 비교용 복사
sy_prev_c2 = sy_prev_c[sy_shared_cols].copy()
sy_curr_c2 = sy_curr_c[sy_shared_cols].copy()

for col in sy_shared_cols:
    try:
        sy_prev_c2[col] = sy_prev_c2[col].astype(str).str.strip().replace({"": np.nan})
        sy_curr_c2[col] = sy_curr_c2[col].astype(str).str.strip().replace({"": np.nan})
    except:
        pass

# NaN 동일 처리
sy_equal_mask = (sy_prev_c2 == sy_curr_c2) | (sy_prev_c2.isna() & sy_curr_c2.isna())
sy_changed_mask = ~sy_equal_mask

# 셀3과 동일한 기준: 6개 컬럼만 변경 여부 반영
sy_imp_cols = [
    "Last Free Date",
    "P/U APPT DATE",
    "P/U APPT TIME",
    "DELIVERY DATE",
    "Return Date",
    "Container Remark",
]
sy_imp_cols = [c for c in sy_imp_cols if c in sy_shared_cols]

sy_imp_mask = sy_changed_mask[sy_imp_cols]

# 하나라도 바뀐 컨테이너만 필터
sy_row_changed = sy_imp_mask.any(axis=1)

sy_changed_prev = sy_prev_c.loc[sy_row_changed]
sy_changed_curr = sy_curr_c.loc[sy_row_changed]

print(f"변경된 컨테이너 수: {len(sy_changed_curr)} 건\n")

# =========================================================
# 2) 표 만들기 — strip 기준 curr 값 사용
# =========================================================

sy_table = sy_changed_curr.reset_index()

# =========================================================
# 3) Delivery 파싱
# =========================================================

def sy_parse_delivery(raw, base_year):
    if raw is None or (isinstance(raw, float) and np.isnan(raw)):
        return None, None, None

    s0 = str(raw)
    s = s0.strip()
    if s == "":
        return None, None, None

    up = s.upper()

    # MT로 시작하면 제외
    if up.startswith("MT"):
        return None, None, None

    # 날짜 패턴
    date_matches = re.findall(r"(\d{1,2})/(\d{1,2})(?:/(\d{2,4}))?", up)

    # OR 또는 다중 날짜 → 예외
    if len(date_matches) >= 2 or "OR" in up:
        return None, None, s0.strip()

    # 단일 날짜 + DEL
    if len(date_matches) == 1 and "DEL" in up:
        mm, dd, yy = date_matches[0]
        mm = int(mm)
        dd = int(dd)
        if yy:
            yy_i = int(yy)
            if yy_i < 100:
                yy_i += 2000
            year = yy_i
        else:
            year = base_year

        try:
            d = date(year, mm, dd)
        except:
            return None, None, s0.strip()

        tag = "DEL-D" if "DIRECT" in up else "DEL"
        return d, tag, None

    # yyyy-mm-dd 형태 날짜
    try:
        dt = pd.to_datetime(s0, errors="raise")
        return dt.date(), None, None
    except:
        pass

    # 그 외 텍스트
    return None, None, s0.strip()


sy_base_year = sy_drayage_curr_date.year

sy_delivery_dates = []
sy_delivery_texts = []

for v in sy_table["Container Remark"]:
    d, tag, ex = sy_parse_delivery(v, sy_base_year)
    if d is not None:
        base = d.strftime("%Y-%m-%d")
        if tag == "DEL-D":
            sy_delivery_texts.append(f"{base} (DEL-D)")
        elif tag == "DEL":
            sy_delivery_texts.append(f"{base} (DEL)")
        else:
            sy_delivery_texts.append(base)
        sy_delivery_dates.append(d)
    elif ex is not None:
        sy_delivery_texts.append(ex)
        sy_delivery_dates.append(None)
    else:
        sy_delivery_texts.append("")
        sy_delivery_dates.append(None)

# =========================================================
# 4) 표 구성
# =========================================================

sy_changed_table = pd.DataFrame({
    "PO#":       sy_table["Customer Reference No."],
    "MBL#":      sy_table["Master B/L No."],
    "CNTR#":     sy_table["Container No."],
    "LFD":       sy_table["Last Free Date"],
    "PU Date":   sy_table["P/U APPT DATE"],
    "PU Time":   sy_table["P/U APPT TIME"],
    "Delivery":  sy_delivery_texts,
    "Delivered": sy_table["DELIVERY DATE"],
    "Return":    sy_table["Return Date"],
})

# 날짜/시간 표시 변환
for col in ["LFD", "PU Date", "Delivered", "Return"]:
    dt = pd.to_datetime(sy_changed_table[col], errors="coerce")
    sy_changed_table[col] = dt.dt.strftime("%Y-%m-%d").fillna("")

# 시간
sy_changed_table["PU Time"] = pd.to_datetime(
    sy_changed_table["PU Time"], errors="coerce"
).dt.strftime("%H:%M").fillna("")

# 정렬용
sy_changed_table["Delivery_sort"] = pd.to_datetime(sy_delivery_dates, errors="coerce")
sy_changed_table["LFD_sort"] = pd.to_datetime(sy_changed_table["LFD"], errors="coerce")

# 정렬
sy_changed_table = (
    sy_changed_table
    .sort_values(["LFD_sort", "Delivery_sort"], ascending=[True, True])
    .drop(columns=["LFD_sort", "Delivery_sort"])
    .reset_index(drop=True)
)

sy_changed_table.index = range(1, len(sy_changed_table) + 1)

# =========================================================
# 5) 하이라이트 마스크
# =========================================================

sy_mask_for_style = pd.DataFrame(False, index=sy_changed_table.index, columns=sy_changed_table.columns)

# strip 비교 마스크 그대로 사용
sy_mask_src = sy_changed_mask[sy_imp_cols].loc[sy_row_changed]

sy_col_map = {
    "Last Free Date":   "LFD",
    "P/U APPT DATE":    "PU Date",
    "P/U APPT TIME":    "PU Time",
    "Container Remark": "Delivery",
    "DELIVERY DATE":    "Delivered",
    "Return Date":      "Return",
}

for idx, row in sy_changed_table.iterrows():
    key = (row["PO#"], row["CNTR#"])
    if key not in sy_mask_src.index:
        continue

    src = sy_mask_src.loc[key]

    for src_col, disp_col in sy_col_map.items():
        if src_col in src.index and disp_col in sy_mask_for_style.columns:
            # 변경된 셀인데 Delivery 표시가 빈칸이면 색칠 X
            if src_col == "Container Remark":
                if str(row["Delivery"]).strip() == "":
                    continue

            if bool(src[src_col]):
                sy_mask_for_style.at[idx, disp_col] = True

# =========================================================
# 6) 스타일 출력
# =========================================================

def sy_highlight_changes(df):
    styles = np.where(
        sy_mask_for_style.values,
        "background-color: #fff3b0; color: red;",
        ""
    )
    return pd.DataFrame(styles, index=df.index, columns=df.columns)

sy_styled_table = (
    sy_changed_table.style
        .set_table_styles([{"selector": "th", "props": [("text-align", "center")]}])
        .apply(sy_highlight_changes, axis=None)
)

sy_styled_table


변경된 컨테이너 수: 65 건



,PO#,MBL#,CNTR#,LFD,PU Date,PU Time,Delivery,Delivered,Return
1,25-6053,SMLMSEL5K8202600,SMCU1163209,2025-11-14,2025-11-14,23:00,,2025-11-17,2025-11-18
2,25-6051,SMLMSEL5K8966200,SMCU1229899,2025-11-14,2025-11-14,23:00,,2025-11-17,2025-11-18
3,25-5954,KORPPNC4658070,TEMU8979328,2025-11-17,2025-11-17,07:00,,2025-11-17,2025-11-21
4,25-6870,KORPPNC4651058,KMTU9354810,2025-11-17,2025-11-17,09:00,,2025-11-17,2025-11-21
5,25-6841,ZIMUSEL71188163,CAAU9554975,2025-11-18,2025-11-18,14:00,,2025-11-19,2025-11-21
6,25-6836,ZIMUSEL71188161,BEAU6334358,2025-11-18,2025-11-18,14:00,,2025-11-19,2025-11-21
7,25-6842,ZIMUSEL71188151,ZCSU7184234,2025-11-18,2025-11-18,15:00,,2025-11-19,2025-11-21
8,25-6843,ZIMUSEL71188143,TGBU7130207,2025-11-18,2025-11-18,14:00,,2025-11-20,2025-11-21
9,25-6828,ZIMUSEL71188158,JXLU4660830,2025-11-18,2025-11-21,11:00,DEL 11/22 OR 11/24,,
10,25-6849,ZIMUSEL71188174,JXLU6168075,2025-11-19,2025-11-20,15:00,2025-11-22 (DEL),,


---

## Save

In [42]:
# ==========================
# sy_drayage 셀 5 — 엑셀로 저장
#  - 정렬: LFD → Delivery_sort
#  - 빈칸 Delivery 색칠 금지
#  - strip 비교 그대로 반영
# ==========================

from pathlib import Path
from openpyxl import load_workbook
from openpyxl.styles import PatternFill, Font, Alignment
import re

# 1) 저장 경로 준비
sy_OUTPUT_DIR = Path("output")
sy_OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

sy_date_str = sy_drayage_curr_date.strftime("%Y%m%d")
sy_output_path = sy_OUTPUT_DIR / f"sy_drayage_diff_{sy_date_str}.xlsx"

# 2) 셀4 결과 복사
sy_df_save = sy_changed_table.copy()

# 3) LFD를 datetime으로 캐스팅 (정렬 안정성)
sy_df_save["LFD"] = pd.to_datetime(sy_df_save["LFD"], errors="coerce")

# 4) Delivery_sort 생성 (YYYY-MM-DD 추출)
def sy_extract_delivery_date(val):
    if not isinstance(val, str):
        return None
    m = re.search(r"\d{4}-\d{2}-\d{2}", val)
    return m.group(0) if m else None

sy_df_save["Delivery_sort"] = pd.to_datetime(
    sy_df_save["Delivery"].apply(sy_extract_delivery_date),
    errors="coerce"
)

# 5) 정렬
sy_df_save = sy_df_save.sort_values(
    by=["LFD", "Delivery_sort"],
    ascending=[True, True],
    na_position="last",
)

# Delivery_sort 제거
sy_export_df = sy_df_save.drop(columns=["Delivery_sort"])

# 6) 엑셀 저장
sy_export_df.to_excel(
    sy_output_path,
    index=True,
    sheet_name="SY_Drayage_Diff"
)

# 7) 하이라이트 적용
wb = load_workbook(sy_output_path)
ws = wb.active

# 헤더 가운데 정렬
for col in range(1, ws.max_column + 1):
    ws.cell(row=1, column=col).alignment = Alignment(horizontal="center", vertical="center")

fill_changed = PatternFill(fill_type="solid", fgColor="FFF3B0")
font_changed = Font(color="FF0000")

# 8) 기존 prev/curr 가져오기 (strip 적용한 동일한 기준!)
sy_prev_k = sy_drayage_df_prev.set_index(sy_drayage_KEY)
sy_curr_k = sy_drayage_df_curr.set_index(sy_drayage_KEY)

# 표시 컬럼 → 원본 컬럼 매핑
sy_display_to_orig = {
    "LFD": "Last Free Date",
    "PU Date": "P/U APPT DATE",
    "PU Time": "P/U APPT TIME",
    "Delivery": "Container Remark",
    "Delivered": "DELIVERY DATE",
    "Return": "Return Date",
}

# 9) 엑셀 각 셀 비교 후 색칠
for excel_row_idx, (df_idx, row) in enumerate(sy_export_df.iterrows(), start=2):

    po = row["PO#"]
    cntr = row["CNTR#"]
    key = (po, cntr)

    # prev/curr 둘 다 없는 key면 스킵
    if key not in sy_prev_k.index or key not in sy_curr_k.index:
        continue

    prev_row = sy_prev_k.loc[key]
    curr_row = sy_curr_k.loc[key]

    for col_offset, col_name in enumerate(sy_export_df.columns, start=2):

        if col_name not in sy_display_to_orig:
            continue

        orig_col = sy_display_to_orig[col_name]

        prev_val = prev_row.get(orig_col, pd.NA)
        curr_val = curr_row.get(orig_col, pd.NA)

        # strip 기준으로 비교
        try:
            prev_s = str(prev_val).strip()
            curr_s = str(curr_val).strip()
        except:
            prev_s = prev_val
            curr_s = curr_val

        same = (
            (prev_s == curr_s) or
            (pd.isna(prev_val) and pd.isna(curr_val))
        )

        # Delivery 빈칸은 색칠 제외
        if col_name == "Delivery":
            if str(row["Delivery"]).strip() == "":
                continue

        if not same:
            cell = ws.cell(row=excel_row_idx, column=col_offset)
            cell.fill = fill_changed
            cell.font = font_changed

# 10) 열 너비 자동 조정
for col_cells in ws.columns:
    max_len = 0
    col_letter = col_cells[0].column_letter
    for cell in col_cells:
        if cell.value is None:
            continue
        length = len(str(cell.value))
        if length > max_len:
            max_len = length
    ws.column_dimensions[col_letter].width = max_len + 2

wb.save(sy_output_path)
print(f"저장 완료: {sy_output_path}")


저장 완료: output/sy_drayage_diff_20251121.xlsx
